In [1]:
!pip install langchain-community
!pip install tiktoken
!pip install faiss-cpu
!pip install pdfplumber pandas
!pip install pypdf
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.59.4
    Uninstalling openai-1.59.4:
      Successfully uninstall

In [2]:
import pdfplumber
import pandas as pd

# Initialize a list to hold the data
text_data = []

# Open the PDF file
with pdfplumber.open("/content/BELALANG DENGAN SEMUT.pdf") as pdf:
    for page in pdf.pages:
        # Extract text from each page
        text = page.extract_text()
        if text:  # Check if text was extracted
            text_data.append(text)  # Append the entire text of the page

# Create a DataFrame with each page's text in a separate row
df = pd.DataFrame(text_data, columns=["Text"])

# If you want to concatenate all text into a single string, you can do this:
complete_text = "\n".join(text_data)
single_row_df = pd.DataFrame({"Complete Text": [complete_text]})

Vector Store - These are like special filing cabinets for the number patterns (vectors) created by embedding models. When you want to find similar text later, the vector store helps you quickly find vectors that are close to each other. It's like organizing books by topic, but with numbers instead.

In [3]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv
import os

In [6]:
os.environ["OPENAI_API_KEY"] = '#'

In [7]:
load_dotenv()

False

In [8]:
# Load the document, split it into chunks, embed each chunk and load it into the vector store.
# Insert csv file into raw_document
# Load the PDF document
loader = PyPDFLoader("/content/BELALANG DENGAN SEMUT.pdf")
raw_documents = loader.load()

# Split the raw documents into smaller chunks for processing
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

# Create a vector store using FAISS with the embedded documents
db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [15]:
query = "Tell me what happens to ant in english"
docs = db.similarity_search(query)
print(docs[0].page_content)

KOLEKSI CERITA KANAK-KANAK 
rohayatifasilinus@gmail.com ms  5 
 
  Setiap hari semut keluar mencari bekalan. 
Semut menyimpan makanan itu. Semut 
menyimpannya di dalam lubang kayu.


In [16]:
print(documents[1])

page_content='KOLEKSI CERITA KANAK-KANAK 
rohayatifasilinus@gmail.com ms  2 
 
TAJUK: Belalang dan Semut 
  
 
 Belalang berkawan baik dengan semut. 
Mereka bermain bersama-sama. Malah, 
makan pun bersama-sama.' metadata={'source': '/content/BELALANG DENGAN SEMUT.pdf', 'page': 1}


Retrievers - These are like smart librarians. When you ask a question, they go to the vector store, find the most relevant information, and bring it back. They help fetch the most useful pieces of information based on what you're looking for.

In [17]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

retriever = db.as_retriever()

In [18]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)

query = "Ceritakan apa yang semut buat pada setiap pagi?"
qa.run(query)

<ipython-input-18-067d2fa1133a>:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)
<ipython-input-18-067d2fa1133a>:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run(query)


' Setiap pagi, semut keluar mencari bekalan dan menyimpannya di dalam lubang kayu.'

In [19]:
query = "Kenapa semut berkawan dengan belalang?"
qa.run(query)

' Semut berkawan dengan belalang karena semut ingin mencari biji kacang dan belalang tidak mau pergi karena ingin tidur. Namun, belalang menyesal dan tidak ingin menjadi pemalas lagi setelah dimarahi oleh semut.'

In [20]:
query = "Apa yang berlaku kepada belalang? Terjemah dalam bahasa inggeris?"
qa.run(query)

' The ant invited the grasshopper to search for peanuts. The grasshopper refused to go and wanted to sleep. The grasshopper felt ashamed and regretful and since then, the grasshopper did not want to be lazy anymore. The grasshopper pleaded with the ant but the ant got angry and told the grasshopper to go home and sleep.'

In [21]:
query = "Apa kejadian yang berlaku pada semut pada pertengahan cerita?"
qa.run(query)

' Semut mengajak belalang mencari biji kacang tetapi belalang menolak dan ingin tidur.'

In [22]:
query = "Apa kejadian yang berlaku pada semut pada pertengahan cerita? terjemah dalam bahasa inggeris"
qa.run(query)

' The ant became angry with the grasshopper and told it to go home and sleep.'